## Load 2020 WIDE-formatted ESG data (Generic)

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### Initially developed using the Royal Dutch Shell plc Sustainability Report 2020 report (Many Sheets)

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import sys

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

import sys
sys.path.append('../src/')

In [ ]:
import re
import pandas as pd
import numpy as np

import openpyxl
from openpyxl import load_workbook
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, Font
from itertools import islice

import pint
import pint_pandas
import iam_units
from openscm_units import unit_registry
pint_pandas.PintType.ureg = unit_registry
ureg = unit_registry
ureg.define('fraction = [] = frac')
ureg.define('percent = 1e-2 frac = pct = percentage')
ureg.define('ppm = 1e-6 fraction')

ureg.define("USD = [currency]")
ureg.define("EUR = nan USD")
ureg.define("JPY = nan USD")
ureg.define("MM_USD = 1000000 USD")
ureg.define("revenue = USD")

ureg.define("btu = Btu")
ureg.define("tBtu = T Btu")
ureg.define("boe = 5.712 GJ")
ureg.define("UEDCTM = [shell_index]")

ureg.define("CO2e = CO2 = CO2eq = CO2_eq")
ureg.define("HFC = [ HFC_emissions ]")
ureg.define("PFC = [ PFC_emissions ]")
ureg.define("mercury = Hg = Mercury")
ureg.define("PM10 = [ PM10_emissions ]")

ureg.define("production = [ output ]")
ureg.define("Index = pct = Share")

ureg.define("Number = dimensionless")

one_co2 = ureg("CO2e")
print(one_co2)

from osc_ingest_trino import *
import pyarrow as pa
import pyarrow.parquet as pq
import json
import io
import uuid

In [ ]:
ureg("tonnes CO2e/revenue")

For spreadsheets in WIDE format, pre-process the spreadsheet as a workbook, cascading label data into 3rd-normal form row and column metadata

* var_col is the label of the variable being measured (whose specificity (like CO2, CH4, NOx, etc) often affects units)
* units_col is the column where units are stated
* val_col:last_val_col are the column where the values are quantitatively reported
* last_val_col+1:last_col are additional columns that are presumed to be metadata labels (such as GRI or SASB labels)

We add:
* notes_col (source worksheet-specific; could act as a kind of source table metadata)
* topic_col (sheet-level category; if we wanted large tables, they could be named by topic)
* category_col (to which row-level data rolls up; if we wanted small tables, they could be named by topic:category)
* segment_col (the dimension by which row-level data is segmented)
* units_col (if not already existing in input)

Some spreadsheets use color to express a multi-level category hierarchy (such as Energy Consumption>>Business Use>>Fuel Type).  We concatenate the categories from left to right as the category for our purposes, except we split off the rightmost subcategory as the segmentation.

Based on all of the above, we don't really have table-level metadata other than notes attached to sheets and generic column information.  An argument could be made that we need to allocate specifier columns for additional data we want to split out from our variables.  That could look like:

* spec1_col
* spec2_col

etc

In [ ]:
# var_col = 1

# Magic knowledge
# last_col = 4
# max_hidden_col = 5
# year_regex = r'^(20\d\d) Data$'

ingest_columns = [ 'Variable', 'Notes', 'Topic', 'Category', 'Segmentation', 'Unit' ]
ingest_col_offsets = dict((j,i) for i,j in enumerate(ingest_columns[1:], start=1))

# In this case, Value columns are named like 2020, 2019, 2018, ... .  It is the pd.melt function that gives us an actual Value column.
# the val_col index merely refers to the first such value row (which hopefully has tasty data)

# Magic knowledge
# val_col = units_col+1     # units_col starts as var_col+1, val_col starts as var_col+2 which is also units_col+1

# If topic_row is None, set topic based on name of sheet
# topic = topic_row = None
# header_row = None

# If init_header_row is None, find header row based on color scheme
# init_header_row = 1

class corp_report_magic:
    def __init__(self, shortname, input_filename, ws_start, ws_end, var_col=None, units_col=None, 
                 notes_col=None, topic_row=None, topic_col=None, category_col=None, init_header_row=None, header_row_list=None,
                 header_color=None, cat_color_dict={ None:0 }, year_regex=None, max_hidden_col=None,
                 val_col=None, last_val_col=None):
        self.shortname = shortname
        self.input_filename = '/'.join([os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src')),
                                        'osc-ingest-shell/data/external', input_filename])
        self.ws_start = ws_start,
        self.ws_end = ws_end,
        self.init_topic_row = topic_row    # If topic_row is None, use the worksheet name as the topic
        self.var_col = var_col or 1
        self.init_units_col = units_col            # If units_col is none, we have to allocate it
        self.init_topic_col = topic_col    # If topic_col is non-null, we get topics from this row
        self.init_category_col = category_col
        self.init_notes_col = notes_col
        self.init_val_col = val_col
        self.init_last_val_col = last_val_col
        # val_col, last_val_col, and last_col can be derived from the spreadsheet
        self.units_row = -1 if units_col==None else 0   # -1: Carry across only; 0: no units seen yet; > 0 row of prevailing unit
        self.init_header_row = init_header_row
        self.header_row_list = header_row_list if header_row_list else ([-1] * ws_start) + ([init_header_row] * (ws_end-ws_start+1))
        self.header_row = None
        self.header_color = header_color
        self.cat_color_dict = cat_color_dict
        self.year_regex = year_regex
        # For AEP, there are several hidden columns on the first sheet we must delete
        # to make that sheet line up with other sheets
        self.max_hidden_col = max_hidden_col
        
        self.units_col = units_col
        self.topic_col = topic_col
        self.category_col = category_col
        self.notes_col = notes_col
        self.segmentation_col = None
        self.val_col = val_col or units_col+1 if units_col else var_col+1 if var_col else 2
        self.last_val_col = last_val_col
        self.last_val_row = None              # Set by preprocess (after we've identified our value columns)
        self.last_col = None                  # Set by crop_sheet
    
    def preprocess(self):
        self.wb_superscripts = None
        self.topic_row = self.init_topic_row
        self.units_col = self.init_units_col
        self.topic_col = self.init_topic_col
        self.category_col = self.init_category_col
        self.notes_col = self.init_notes_col
        self.segmentation_col = None
        self.val_col = self.init_val_col or self.init_units_col+1 if self.init_units_col else self.var_col+1
        self.last_val_col = self.init_last_val_col

Shell_magic = corp_report_magic("Shell", r"greenhouse-gas-and-energy-data-shell-sr20.xlsx", 1, 10,
                                init_header_row=5, units_col=2)
DPDHL_magic = corp_report_magic("DPDHL", r"DPDHL-ESG-Statbook-2020-en.xlsx", 2, 4,
                                topic_row=1, header_row_list=[ -1, -1, 8, 5, 4], header_color='FFBF00',
                                cat_color_dict={ 'FF00B050':0, 'E2F0D9':1, 'D0CECE':0, 'E7E6E6':0 },
                                units_col=2)
Unilever_magic = corp_report_magic("Unilever", r"Unilever sustainability performance data_Climate FINAL.xlsx", 0, 0,
                                   topic_row=9, init_header_row=10,
                                   cat_color_dict={'FFEBF1DE':0, 'E2F0D9':0, 'FFFFFF':2})
AEP_magic = corp_report_magic("AEP", r"2021-Data-Centerv1.xlsx", 0, 3,
                              init_header_row=1,
                              cat_color_dict={'FF237F2E':0, 'FF40B14B':1, 'FFC6E7C8':2,
                                              'FF757575':0, 'FFBDBDBD':1, 'FFE5E5E5':2,
                                              'FF5FB3F9':0, 'FFB9DDFC':1, 'FFE2F2FE':2,
                                              'FFD0AF8F':0, 'FFEEDCCA':1, 'FFF2E8DE':2 },
                              year_regex=r'^(20\d\d) Data$', max_hidden_col=5)
Altria_magic = corp_report_magic("Altra", r"esg-tables.xlsx", 1, 1,
                                 init_header_row=2,
                                 cat_color_dict={'FF9BDA44':0, 'FF92D050':1, 'FFE2F0D9':2, 'FFFFFFFF':2},
                                 units_col=2)

SUEZ_magic = corp_report_magic("SUEZ", r"SUEZ-FY-2020-ESG-dataset-xls-may2020.xlsx", 1, 1,
                               init_header_row=3,
                               topic_col=1, category_col=2, var_col=3, units_col=4,
                               val_col=9, last_val_col=10)

filename_magic = {
    r"greenhouse-gas-and-energy-data-shell-sr20.xlsx": Shell_magic,
    r"2021-Data-Centerv1.xlsx": AEP_magic,
    r"DPDHL-ESG-Statbook-2020-en.xlsx": DPDHL_magic,
    r"Unilever sustainability performance data_Climate FINAL.xlsx": Unilever_magic,
    r"esg-tables.xlsx": Altria_magic,
}
# A storage area in case we delete items from the above.
foo = {
    r"esg-tables.xlsx": Altria_magic,
    r"greenhouse-gas-and-energy-data-shell-sr20.xlsx": Shell_magic,
    r"DPDHL-ESG-Statbook-2020-en.xlsx": DPDHL_magic,
    r"Unilever sustainability performance data_Climate FINAL.xlsx": Unilever_magic,
    r"2021-Data-Centerv1.xlsx": AEP_magic,
}

crm = None
value_vars = None

In [ ]:
scale_regex = re.compile(r'^((mi|bi|tri|quadri)llion|thousand|hundred)(s of)? ', flags=re.I)
sc_xlate = {'hun':1e2, 'tho':1e3, 'mil':1e6, 'bil':1e9, 'tri':1e12, 'qua':1e15}

def find_units(var):
    scale = 1.0
    if var in ['%', 'pct', 'percent']:
        return 'percent'
    if '-based' in var or 'KPI' in var:
        return None
    if 'Total no.' in var:
        var = var.replace('Total no.', 'Number')
    elif 'No.' in var:
        var = var.replace('No.', 'Number')
    if ' of production' in var:
        var = var.replace(' of production', '')
    var = var.replace ('Net MWh', 'MWh')
    var = var.replace ('trillion (10^12)', 'trillion')
    var = var.replace ('m3', 'kl')
    var = var.replace ('KWh', 'kWh')
    var = var.replace ('Index points', 'Number')
    if var in ureg:
        return f'{ureg(var).u:~}'
    if var.lower() in ureg:
        return f'{ureg(var.lower()).u:~}'
    m = re.search(r'(((metric)|(short)) t)on', var, flags=re.I)
    if m:
        var = '_'.join(var.lower().split(' '))
        if var in ureg:
            return f'{ureg(var).u:~}'
    m = re.search(scale_regex, var)
    if m:
        var = ' '.join([var[0:m.start(0)],var[m.end(0):]]).strip()
        if var in ureg:
            units = sc_xlate[m.group(1)[0:3].lower()] * ureg(var)
            units = units.to_compact()
            if units.m - 1.0 < 0.00001:
                # Address roundoff problems such as giga = 1.00000000000002 x 10^9
                return f'{units.u:~}'
            print(f'units do not reduce: {units}')
    print(f'find units: nothing found for {var}')
    return None

In [ ]:
topic_keywords = { 'footprint':['intensity'],
                   'emissions':['scope 1', 'scope 2', 'scope 3', 'ghg', 'intensity'],
                   'energy':['consum', 'generat', 'renewable', 'intensity'],
                   'water':['consum', 'discharge', 'withdraw', 'intensity'],
                   'waste':['landfill', 'incinerate', 'compost', 'recycle', 'reuse', 'intensity'],
                   'other':[]}

topic_cell = None
category_cell = None
segmentation_stack = []

In [ ]:
def process_topic(ws, row):
    """
    Topics are major headers.  If such headers also have units defined, they are also categories.
    If such headers also have values defined, they are also processed as a variable.
    """
    global topic_cell, category_cell
    
    if row==None:
        # We have to put the topic in header_row+1 because header_row is column info, not data, for the dataframe
        row = crm.header_row+1
        topic_cell = ws.cell(row, crm.topic_col)
        topic_cell.value = ws.title.lower()
        print(f'process_topic {row}: setting topic from title {topic_cell.value}')

    cell = ws.cell(row, crm.var_col)
    if cell.value:
        var_text = cell.value.split('\n')[0]    # notes have been stripped out
    else:
        var_text = None
    
    if var_text:
        topic_cell = ws.cell(row, crm.topic_col)
        if topic_cell.value!=ws.title.lower():
            # Let's assume topic text is not parenthetical, but titular
            var_text = re.sub(r'\(.+\)', '', var_text)
            var_words = var_text.split(' ')
            for word in var_words:
                if topic_cell.value and word.lower() == topic_cell.value:
                    # Not a new topic
                    break
                if word.lower() in topic_keywords.keys():
                    print(f'process_topic {row}: setting topic {word}')
                    topic_cell.value = word.lower()

            if topic_cell.value==None:
                print(f'worksheet {ws.title}: unknown topic {var_text}')
                topic_cell.value = ws.title.lower()
                # topic_keywords[ws.title.lower()] = []

        # Try to extract units from Variable description
        if ws.cell(row, crm.units_col).value==None:
            p_exprs = re.findall(r'\((.+)\)', ws.cell(row, crm.var_col).value)
            for p in p_exprs:
                if find_units(p):
                    print(f'process_topic {row}: setting units from var: {p}')
                    ws.cell(row, crm.units_col).value = p
                    break
        # Here we don't look for species_unit; mistake???
        
        # If we definitely have units, set the category, which will also process the variable (if needed)
        if ws.cell(row, crm.units_col).value:
            print(f'process_topic {row}: setting category {var_text}')
            category_cell = ws.cell(row, crm.category_col)
            ws.cell(row, crm.category_col).value = var_text

            print(f'process_topic {row}: setting units {ws.cell(row, crm.units_col).value}')
            units = find_units (ws.cell(row, crm.units_col).value)
            if units == None:
                error(f'unknown units {ws.cell(row, crm.units_col).value}')
            ws.cell(row, crm.units_col).value = units
            
            row = process_categories (ws, row)
    else:
        print(f'process_topic {row}: no var text')
    if row < 0 or row >= crm.last_val_row:
        return crm.last_val_row
    return row+1

In [ ]:
def formatted_as_sub(cell1, cell2):
    color1 = cell2rgb(cell1)
    color2 = cell2rgb(cell2)
    if color1 in crm.cat_color_dict:
        if color2 in crm.cat_color_dict:
            if crm.cat_color_dict[color1] < crm.cat_color_dict[color2]:
                return True
            if crm.cat_color_dict[color1] > crm.cat_color_dict[color2]:
                return False
        else:
            return True
    elif color2 in crm.cat_color_dict:
        return False
    
    sub_score = 0
    if cell1.font.b and cell2.font.b==False:
        # print('+bold')
        sub_score += 1
    if cell1.font.b==False and cell2.font.b:
        # print('-bold')
        sub_score -= 1
    if cell1.font.u and cell2.font.u==False:
        # print('+underline')
        sub_score += 1
    if cell1.font.u==False and cell2.font.u:
        # print('-underline')
        sub_score -= 1
    if cell1.alignment.indent < cell2.alignment.indent:
        # print('+indent')
        sub_score += 1
    elif cell1.alignment.indent > cell2.alignment.indent:
        # print('-indent')
        sub_score -= 1
    if cell1.font.sz < cell2.font.sz:
        # print('+size')
        sub_score += 1
    elif cell1.font.sz > cell2.font.sz:
        # print('-size')
        sub_score -= 1
    if cell1.alignment.horizontal == 'left' and cell2.alignment.horizontal == 'right':
        # print('+halign')
        sub_score += 1
    elif cell1.alignment.horizontal == 'right' and cell2.alignment.horizontal == 'left':
        # print('-halign')
        sub_score -= 1
    # print(f'sub_score = {sub_score}')
    if sub_score > 0:
        return True
    if sub_score < 0:
        return False
    if sub_score == 0:
        return None

def process_categories(ws, row):
    """
    Categories have units
    """
    global topic_cell, category_cell, segmentation_stack
    
    while row < crm.last_val_row:
        cell = ws.cell(row, crm.var_col)
        if cell.value:
            var_text = cell.value.split('\n')[0]    # notes have been stripped out
        else:
            # Some categories are declared across multiple lines, with units by themselves
            var_text = None
        
        # If we're already processing this row as a topic (and being called from that context), don't recurse
        if var_text and topic_cell.row < row:
            color = cell2rgb(ws.cell(row,1))
            if color and color in crm.cat_color_dict:
                # Hack because Unilever uses colors wrongly, but punctuation saves the day
                if crm.cat_color_dict[color]==0 and var_text[-1]!=':':
                    # Register that we have a new topic
                    topic_cell = ws.cell(row, crm.topic_col)
                    topic_cell.value = var_text
                    print(f'process_categories: new topic set at row {row}: {var_text} (color = {color})')
                    return process_topic(ws, row)
                if crm.cat_color_dict[color]==1:
                    category_cell = ws.cell(row, crm.category_col)
                    category_cell.value = var_text
                    print(f'process_categories: new category set at row {row}: {var_text} (color = {color})')
            elif color:
                if color!='00000000':
                    print(f'process_categories: unknown color {color} at row {row}: {var_text}')
            elif None in crm.cat_color_dict:
                category_cell = ws.cell(row, crm.category_col)
                category_cell.value = var_text
                print(f'process_categories: new category set at row {row}: {var_text}')

        # Try to extract units from Variable description
        if ws.cell(row, crm.units_col).value==None and var_text:
            p_exprs = re.findall(r'\((.+)\)', ws.cell(row, crm.var_col).value)
            for p in p_exprs:
                if 'scope' in p.lower() or 'category' in p.lower():
                    continue
                if find_units(p):
                    print(f'process_categories {row}: setting units from var: {p}')
                    ws.cell(row, crm.units_col).value = p
                    break
        
        # Apply our best guess for units in case we need to propagate in segmentation
        var_units = ws.cell(row, crm.units_col).value
        var_species = ''
        if var_units:
            var_units = find_units (var_units)
            m = re.search('r\((.+)\)', var_text)
            if m:
                var_species = m.group(1)
                species_units = find_units(' '.join([var_units, var_species]))
                if species_units:
                    units = species_units
                    var_text = ' '.join([var_text[0:m.start(1)], var_text[m.end(1)+1:]]).replace('  ', ' ')
            else:
                units = var_units
        elif category_cell:
            units = ws.cell(category_cell.row, crm.units_col).value
        else:
            units = None
        ws.cell(row, crm.units_col).value = units
        
        total_of = ''
        if var_text and 'total' in var_text.lower():
            c1, c2 = re.split(r'\s*totals?\s*', var_text, flags=re.I)
            if c2.strip()=='':
                total_of = c2 = c1
                c1 = var_text
            elif c1.strip()=='':
                c1 = var_text
                total_of = c2
            else:
                total_of = var_text
                # we have c1:c2
        
        segment_by = ''
        for x in [ ' per ', ' by ', ' of ' ]:
            if var_text and x in var_text:
                segment_by = x
                break

        if formatted_as_sub(ws.cell(row, crm.var_col), ws.cell(row+1, crm.var_col)):
            if segment_by:
                c1, c2 = var_text.split(segment_by, 1)
            elif not total_of:
                c1 = var_text
                c2 = '(anon)'
            category_cell.value = c1
            if segment_by or total_of:
                print(f'process_categories {row}: segmenting {c1}::{c2}')
                segmentation_stack = [ ws.cell(row, crm.segmentation_col) ]
                segmentation_stack[-1].value = c2
                row = process_var(ws, row)
                row = process_segmentation (ws, row)
                if segmentation_stack != []:
                    print(f'process_categories: segmentation_stack = {segmentation_stack}')
                if row < 0:
                    return row
                continue
        segmentation_stack = []
        # print(f'process_category {row}: processing variable')
        row = process_var(ws, row)
        if row < 0:
            return row
    if row < crm.last_val_row:
        return row+1
    if row == crm.last_val_row:
        process_var (ws, row)
    return -1

In [ ]:
def process_segmentation(ws, row):
    """Process rows starting at ROW as a part of a segmentation.  We push and recurse if we see
    a new level of indentation.  We pop and return if we see an outdent.
    """
    global segmentation_stack
    
    seg_start_cell = ws.cell(row, crm.var_col)
    seg_start_units = ws.cell(row, crm.units_col).value
    while row < crm.last_val_row:
        cell = ws.cell(row, crm.var_col)
        if cell.value==None:
            if ws.cell(row, crm.units_col).value and ws.cell(row+1, crm.units_col).value==None:
                ws.cell(row+1, crm.units_col).value = ws.cell(row, crm.units_col).value
                row = row+1
                # Don't start a segment with an unlabeled cell
                seg_start_cell = ws.cell(row, crm.var_col)
                continue
            else:
                print(f'unhandled units at row {row}: {ws.cell(row+1, crm.units_col).value}; {ws.cell(row, crm.units_col).value}')

        if cell==seg_start_cell:
            # Handle easy case
            row = process_var (ws, row)
        else:
            new_seg_start = formatted_as_sub(seg_start_cell, cell)
            if new_seg_start:
                if ws.cell(row, crm.units_col).value==None:
                    ws.cell(row, crm.units_col).value = seg_start_units
                # There could be many rows at the same level as SEG_START_CELL before a new segmentation is seen
                # The label we care about is the one immediately preceding, not the first one with that indentation
                segmentation_stack.append(ws.cell(row-1, crm.var_col))
                row = process_segmentation(ws, row)
            elif new_seg_start==False:
                segmentation_stack.pop()
                print(f'pop at row {row}: segmentation_stack now {segmentation_stack}')
                return row
            else:
                if ws.cell(row, crm.units_col).value==None and seg_start_units:
                    ws.cell(row, crm.units_col).value = seg_start_units
                row = process_var (ws, row)
        if row < 0:
            return row
    if row < crm.last_val_row:
        return row+1
    if row == crm.last_val_row:
        process_var (ws, row)
        if segmentation_stack:
            print(f'process_segmentation: stack at end = {segmentation_stack}')
            segmentation_stack = []
    return -1

In [ ]:
def process_var(ws, row):
    global topic_cell, category_cell, segmentation_stack
    
    cell = ws.cell(row, crm.var_col)
    
    # Treat X (Y) as 'Category X Segmentation Y'
    if cell.value:
        var_text = cell.value.split('\n')[0]    # notes have been stripped out
        var_text = var_text.replace('(%)', '(pct)')
        m = re.search(r'^(.*) \((.*?)\)', var_text)
    else:
        var_text = None
        m = None
    if m and '-based' not in m.group(2) and 'scope' not in m.group(2).lower() and 'category' not in m.group(2).lower():
        species_units = find_units (m.group(2))
        if species_units:
            # print(f'process_var {row}: found species or units in {var_text}')
            units = ws.cell(row, crm.units_col).value
            if units and units != species_units:
                species_units = find_units(' '.join([units, m.group(2)]))
                if species_units:
                    var_text = m.group(1).rstrip()
                    units = species_units
                else:
                    print(f'??? Not overriding {units} with {m.group(2)}')
                    # units = ws.cell(row, crm.units_col).value
            else:
                units = species_units
                if ' ' not in m.group(1) and m.group(1) in ureg:
                    species_units = find_units(' '.join([units, m.group(1)]))
                    if species_units:
                        var_text = m.group(1).rstrip()
                        units = species_units
                # print(f'Inferring/composing units: {units}')
            if units != ws.cell(category_cell.row, crm.units_col).value:
                # print(f'changing units from category: {ws.cell(category_cell.row, crm.units_col).value} to {units}')
                ws.cell(row, crm.units_col).value = units
        elif m.group(2).lower() and topic_cell.value.lower() in topic_keywords and m.group(2).lower() in topic_keywords[topic_cell.value.lower()]:
            # Scope 1 is actually a sneaky segmentation
            category_cell = ws.cell(row, crm.category_col)
            category_cell.value = m.group(2)
        else:
            print(f'process_var {row}: unhandled ( {m.group(2)} )')
    else:
        if ws.cell(row, crm.units_col).value == None:
            # print(f'process_var {row}: propagating units {ws.cell(category_cell.row, crm.units_col).value}')
            ws.cell(row, crm.units_col).value = ws.cell(category_cell.row, crm.units_col).value
        else:
            # print(f'process_var {row}: using units {ws.cell(row, crm.units_col).value}')
            pass
    ws.cell(row, crm.topic_col).value = topic_cell.value
    ws.cell(row, crm.category_col).value = category_cell.value
    if segmentation_stack != []:
        ws.cell(row, crm.segmentation_col).value = '::'.join(s.value for s in segmentation_stack)
    if row < crm.last_val_row:
        return row+1
    return -1

In [ ]:
# ??? The header row color is going to be spreadsheet-specific.  This is what DPDHL gives us.

import cell2rgb
from cell2rgb import cell2rgb

def find_header_row(wb, ws):
    # If we haven't found the header by max_row-1, we'll never find it...
    for row in range(1, ws.max_row):
        color = cell2rgb(ws.cell(row,1))
        if color == crm.header_color:
            return row
        print(f'find_header_row: color = {color}')
    error('No header found')
    return -1

In [ ]:
import zipfile
from lxml import etree
import xml.etree.ElementTree as eTree

# We pre-process the structure of the worksheet so that it can be trivially loaded into a dataframe for further reshaping.

# Stash notes for each worksheet here.  These are *per worksheet*
# ??? In the case of DPDHL, there's a Comment field we don't track, which means we miss a stated target
ws_notes = {}

def preprocess(wb, ws):
    global crm, value_vars
    global topic_cell, category_cell, segmentation_stack
    
    cell_notes_text = []
    cell_notes_cells = []
    
    def crop_sheet(ws):
        global crm
        # Frist, set max_row/max_column based on actually active cells, not cells with random spaces or empty strings
        this_max_row = 1
        this_max_col = 1
        for row in range(1,ws.max_row+1):
            row_max_col = None
            for col in range(1,ws.max_column+1):
                cell = ws.cell(row,col)
                if cell.value==None:
                    continue
                if type(cell.value)==str and cell.value.strip()=='':
                    cell.value = None
                    continue
                if col > this_max_col:
                    this_max_col = col
                row_max_col = col
            if row_max_col:
                this_max_row = row
        print('crop_sheet')
        print('{} x {}'.format(ws.max_row, ws.max_column))
        ws.delete_rows(this_max_row+1,ws.max_row)
        ws.delete_cols(this_max_col+1,ws.max_column)
        print('{} x {}'.format(ws.max_row, ws.max_column))
        crm.last_col = ws.max_column
        
    def preprocess_notes():
        global crm
        z = zipfile.ZipFile(crm.input_filename)

        if crm.wb_superscripts==None:
            with z.open('xl/sharedStrings.xml') as fp:
                ss_xml = etree.fromstring(fp.read())
            # get the namespaces                                                                                                                                                                                                                                             
            ssns = ss_xml.nsmap
            if None in ssns:
                ssns['none'] = ssns.pop(None)
            crm.text_list = ss_xml.xpath('//none:si', namespaces=ssns)
            # All shared strings across all sheets with superscripts                                                                                                                                                                                                                           
            crm.wb_superscripts = [s for s in range(len(crm.text_list))
                                   if 'superscript' in eTree.tostring(crm.text_list[s], encoding='unicode')]
        
        with z.open(f'xl/worksheets/sheet{wb.worksheets.index(ws)+1}.xml') as fp:
            ws_xml = etree.fromstring(fp.read())
        z.close()

        # get the namespaces                                                                                                                                                                                                                                             
        wsns = ws_xml.nsmap
        if None in wsns:
            wsns['none'] = wsns.pop(None)
        cell_list = ws_xml.xpath('//none:c', namespaces=wsns)

        # Dictionary of cells:shared strings with superscripts within this sheet's cells                                                                                                                                                                                                          
        sheet_ss_dict = {c:s for c in range(len(cell_list))
                         for s in crm.wb_superscripts if f' t="s"><ns0:v>{s}</ns0:v></ns0:c>' in eTree.tostring(cell_list[c],
                                                                                                                 encoding='unicode')}

        cell_notes_text = []
        cell_notes_cells = []
        for c, s in sheet_ss_dict.items():
            cell_name = eTree.tostring(cell_list[c], encoding='unicode').split(' ')[2].split('=')[1][1:-1]
            cell_text_xml = eTree.tostring(crm.text_list[s],encoding='unicode')
            ss_bool = ['superscript' in x for x in cell_text_xml.split('<ns0:r>')[1:]]
            cell_text_parts = [re.split('<ns0:t.*?>', x)[-1].split('</ns0:t>')[0]
                  for x in cell_text_xml.split('<ns0:r>')[1:]]
            for x in range(len(ss_bool)-1):
                if ss_bool[x]==False and ss_bool[x+1]==True:
                    cell_notes_text.append(cell_text_parts[x+1])
                    cell_notes_cells.append(cell_name)
                    ws[cell_name].value = ws[cell_name].value.replace(cell_text_parts[x]+cell_text_parts[x+1],
                                                                      cell_text_parts[x])
    
    crm.preprocess()
    preprocess_notes()
    print(cell_notes_text)
    print(cell_notes_cells)
    
    topic_cell = None
    category_cell = None
    segmentation_stack = []

    # Remove merged cells
    mergedRanges=ws.merged_cells.ranges
    while mergedRanges:
        for entry in mergedRanges:
            ws.unmerge_cells(str(entry))

    if crm.max_hidden_col and wb.worksheets[0]==ws:
        ws.delete_cols(1,crm.max_hidden_col)
    crop_sheet(ws)

    if crm.init_header_row:
        crm.header_row = crm.init_header_row
    else:
        crm.header_row = find_header_row (wb, ws)
    
    # Reset this for each worksheet
    if crm.units_row >= 0:
        crm.units_row = 0

    col = crm.val_col
    last_val_col = crm.last_val_col or col
    while crm.last_val_col==None or col<=crm.last_val_col:
        # ??? Deal with note in header value (such as '2019(b)' or, God forbit '20197' where the superscripted 7 just sits like it's part of the number)
        if crm.year_regex:
            maybe_year = re.sub(crm.year_regex, r'\1', str(ws.cell(crm.header_row, col).value))
        else:
            maybe_year = str(ws.cell(crm.header_row, col).value)
        if len(maybe_year)>=4 and maybe_year[0:2]=='20' and maybe_year[2].isdigit() and maybe_year[3].isdigit():
            ws.cell(crm.header_row, col).value = maybe_year[0:4]
            last_val_col = col
        elif crm.last_val_col==None:
            crm.last_val_col = last_val_col
            break
        col = col+1
    value_vars = [ None ] * (crm.last_val_col-crm.val_col+1)
    for col in range(crm.val_col, crm.last_val_col+1):
        value_vars[col-crm.val_col] = ws.cell(crm.header_row, col).value
    print(value_vars)
    
    # Make space for TOPIC : CATEGORY : SEGMENTATION triple.
    # This triple could very well become an index into a data framework (such as SASB, TCFD, etc)
    new_column_count = (len(ingest_columns)-1
                        -int(crm.notes_col!=None)
                        -int(crm.topic_col!=None)
                        -int(crm.category_col!=None)
                        -int(crm.units_col!=None))
    ws.insert_cols(crm.last_val_col+1,amount=new_column_count)
    if crm.notes_col==None:
        crm.notes_col = crm.last_val_col+ingest_col_offsets['Notes']
    ws.cell(crm.header_row,crm.notes_col).value = 'Notes'
    if crm.topic_col==None:
        crm.topic_col = crm.last_val_col+ingest_col_offsets['Topic']
    ws.cell(crm.header_row,crm.topic_col).value = 'Topic'
    if crm.category_col==None:
        crm.category_col = crm.last_val_col+ingest_col_offsets['Category']
    ws.cell(crm.header_row,crm.category_col).value = 'Category'
    crm.segmentation_col = crm.last_val_col+ingest_col_offsets['Segmentation']
    ws.cell(crm.header_row,crm.segmentation_col).value = 'Segmentation'
    if crm.units_col==None:
        crm.units_col = crm.last_val_col+ingest_col_offsets['Unit']
    ws.cell(crm.header_row,crm.units_col).value = 'Unit'
    ws.cell(crm.header_row,crm.var_col).value = 'Variable'
        
    crm.last_col = crm.last_col + new_column_count

    # Find last row of actual values so we can process notes at the end
    for row in range(ws.max_row, 0, -1):
        if any([True for col in range(crm.val_col, crm.last_val_col+1) if ws.cell(row, col).value]):
            crm.last_val_row = row
            break

In [ ]:
def postprocess(wb, ws):
    
    # Intended for Shell notes
    def save_ws_notes(ws, note):
        global ws_notes
        
        if ws.title not in ws_notes:
            ws_notes[ws.title] = {}
        note_label, note_text = note.split(' ', 1)
        ws_notes[ws.title][note_label] = note_text.strip()
    
    # Intended for DPDHL notes
    def save_ws_notes2(ws, note):
        global ws_notes
        
        if ws.title not in ws_notes:
            ws_notes[ws.title] = {}
        notes = re.split(r' (\d+)\)\s+', note)
        print('NOTES')
        print(notes)
        print('END NOTES')
        ws_notes[ws.title]['0'] = notes[0]
        for i in range(int(len(notes)/2)):
            ws_notes[ws.title][notes[1+2*i]] = notes[2+2*i].strip()
    
    # Used for Unilever
    def finish_notes(row):
        print('finish_notes @ {}'.format(row))

    for row in range(crm.last_val_row+1, ws.max_row+1):
        cell = ws.cell(row, crm.var_col)
        # Find either bracketed note or note that begins with possible superscript
        if cell.value==None:
            continue
        if cell.value[0]=='[':
            save_ws_notes(ws, cell.value)
            continue
        if re.search(r'^[^(]*\d[)]', str(cell.value)):
            save_ws_notes2(ws, cell.value)
        if re.search(r'notes', str(cell.value), flags=re.I):
            finish_notes(row)
            return

In [ ]:
# With a nicely formatted workbook, do the rest of our work (including writing to Trino) using dataframes

# IPIECA, SASB, and GRI columns all feed metadata

def ws_to_df(wb, i):
    data = islice(wb.worksheets[i].values, crm.header_row_list[i]-1, None)
    cols = list(next(data))
    data = list(data)
    # idx = [r[0] for r in data]
    # data = (islice(r, 0, None) for r in data)
    cols[crm.units_col-1] = 'Unit'            # Already set by Shell; DF indexes are XLSX-1
    df = pd.DataFrame(data, columns=cols) # we don't pass in an index here

    # Remove null columns
    df = df[[c for c in df.columns if c!= None]]
    
    # For now, do not remove rows lacking units.  Those are basically where Notes are stored (for better or worse).
    # print('rows lacking proper Units')
    # display(df[df['Unit'].isnull()])
    df = df.loc[df.Unit.notna() | df.Category.isna()]

    # Clear out data that is n/a, n/c (not collected), n/d (not disclosed)
    df[df['Unit'].notna()].replace(to_replace='^n/[acd]$', value='', regex=True, inplace=True)
    
    # Change numerical years to strings to make pandas indexing behave
    df.columns = [str(c) for c in df.columns]
    # Drop completely empty rows
    # df.dropna(how='all', axis=0, inplace=True)
    return df

Write out polymorphic dataframe in LONG format.  This follows tidy data model, with one variable observation per row.  
Polymorphic means that Units/dimensions of each row are specified, but not necessarily the same row to row.  
Aggregation functions must be careful that selection criteria does not mix up incompatible unit types and/or observation variables.

In [ ]:
wb = None
ws = None
melted_df = None

def ingest_filename(filename):
    global melted_df
    global topic_cell
    global crm, wb, ws
    
    crm = filename_magic[filename]

    wb = load_workbook(crm.input_filename, data_only=True)

    # For a label like "Scope 1 emissions by country" return ['', 'Scope 1 emissions', 'country']
    # For a label like "Direct GHG emissions (Scope 1) [A] [B] [C] [D]" return ['[A] [B] [C] [D]', 'Direct GHG emissions (Scope 1) ', '']

    long_fmt_filename = ''
    wide_fmt_filename = ''

    for i in range(crm.ws_start[0], crm.ws_end[0]+1):
        ws = wb.worksheets[i]
        ws_notes = {}
        preprocess(wb, ws)
        
        row = process_topic(ws, crm.topic_row)

        while row < crm.last_val_row:
            if row == crm.header_row_list[i]:
                row = row+1
                continue
            color = cell2rgb(ws.cell(row,1))
            if color and color in crm.cat_color_dict:
                if crm.cat_color_dict[color]==0:
                    # Register that we have a new topic
                    topic_cell = ws.cell(row, crm.topic_col)
                    topic_cell.value = ws.cell(row, crm.var_col).value
                    print(f'new topic set at row {row}: {topic_cell.value} (color = {color})')
                    row = process_topic(ws, row)
                else:
                    row = process_categories(ws, row)
                    if row < 0:
                        break
            else:
                print(f'ingest_file: processing {row}')
                row = process_topic(ws, row)
    
        # process_topic (wb, ws, header_row_list[i])
        # preprocess2(wb, ws)
        postprocess(wb, ws)
        # What to do with ws_notes???
        df = ws_to_df(wb, i)
        df.replace('',pd.NA,inplace=True)
        print(f'wb({i}) dataframe')
        # display(df.loc[0:min(len(df),45)])
        melted_df = pd.melt(df, id_vars=ingest_columns, var_name='Year', value_name='Value', value_vars=value_vars)
        melted_df.dropna(subset=['Value'],inplace=True)
        melted_df = melted_df.astype({'Year': 'int'})

        if i==crm.ws_start[0]:
            report_year = max(df.columns[crm.val_col-1:crm.last_val_col])
            long_fmt_filename = ''.join([os.environ.get('PWD', '/opt/app-root/src'), '/osc-ingest-shell/data/interim/',
                                         crm.shortname, '_', report_year, '_', 'LONG.xlsx'])
            writer_long = pd.ExcelWriter(long_fmt_filename)
            wide_fmt_filename = ''.join([os.environ.get('PWD', '/opt/app-root/src'), '/osc-ingest-shell/data/interim/',
                                         crm.shortname, '_', report_year, '_', 'WIDE.xlsx'])
            writer_wide = pd.ExcelWriter(wide_fmt_filename)

        # This writes out LONG data with TOPIC as SHEET_NAME.  Later we'll create a truly long table with TOPIC restored as a column
        melted_df.loc[:, melted_df.columns != 'Topic'].to_excel(writer_long, index=False, sheet_name=df.iloc[0]['Topic'][0:30])

        print(ws.title)
        columns = ['Variable', 'Unit']
        # We need these columns to reshape our data
        for extra_col in ['Notes', 'Category', 'Segmentation']:
            if df[extra_col].notna().any():
                columns.append(extra_col)
        # In the case of Shell, we have only one topic per sheet, so can transform melted_df directly
        pf = melted_df.pivot(index=['Year', 'Topic'], columns=columns, values=['Value'])
        pf = pf.droplevel('Topic')
        # Once reshaped, the extra columns actually appear as multi-level indexes.  Drop them from also behaving like values
        pf[[c for c in columns if c not in ['Variable', 'Unit']]] = pd.NA
        pf.dropna(how='all', axis=1, inplace=True)
        pf.to_excel(writer_wide, sheet_name=df.iloc[0]['Topic'][0:30])

    writer_long.close()
    writer_wide.close()
    
    # We are now working with our own workbook, which doesn't have a zero-index sheet to ignore
    # Make the workbook more legible to those reading it
    long_wb = load_workbook(long_fmt_filename, data_only=True)
    for ws in long_wb.worksheets:
        dim_holder = DimensionHolder(worksheet=ws)
        for col in range(ws.min_column, ws.max_column + 1):
            if get_column_letter(col)=='A':
                width = 40
            elif get_column_letter(col) in ['B', 'E']:
                width = 15
            elif get_column_letter(col) in ['C', 'D']:
                width = 25
            else:
                width = 10
            dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=width)
        ws.column_dimensions = dim_holder
    
    long_wb.save(long_fmt_filename)
    long_wb.close()
    
    def as_text(value):
        if value is None:
            return ""
        return str(value)
    
    # Write out dataframe in WIDE format.  This data is technically tidy, with one multi-dimensional observation per row.
    # Units/dimensions are consistent on a per-column basis, making it easy to aggregate column-based data.
    wide_wb = load_workbook(wide_fmt_filename, data_only=True)
    # Make the workbook more legible to those reading it
    for ws in wide_wb.worksheets:
        dim_holder = DimensionHolder(worksheet=ws)
        for col in range(ws.min_column, ws.max_column + 1):
            cell = ws.cell(2, col)
            cell.alignment = Alignment(wrap_text=True,vertical='top') 
            dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=max(10,1+len(as_text(cell.value))/3))
        ws.column_dimensions = dim_holder

    wide_wb.save(wide_fmt_filename)
    wide_wb.close()

In [ ]:
for filename in filename_magic:
    print(filename)
    crm = filename_magic[filename]
    ingest_filename(filename)

print('Done!')

In [ ]:
segmentation_stack

In [ ]:
ws.cell(11,1).border.top.style

In [ ]:
columns = ['Variable', 'Unit']
# We need these columns to reshape our data
for extra_col in ['Notes', 'Category', 'Segmentation']:
    if melted_df[extra_col].notna().any():
        columns.append(extra_col)
melted_df[melted_df['Segmentation']=='(anon)::Road transport']

In [ ]:
units = 1e6 * ureg('kl/year')
units.to_compact()

In [ ]:
bletch!

In [ ]:
crm = filename_magic['DPDHL-ESG-Statbook-2020-en.xlsx']
wb = load_workbook(crm.input_filename, data_only=True)
ws = wb.worksheets[2]
preprocess(wb, ws)

row = process_topic(pc, crm.topic_row)


In [ ]:
def preprocess2(wb, ws):
    global crm
    
    scope1_gases = ['CO2', 'CH4', 'N2O', 'HFC', 'SF6', 'PFC', 'NF3', 'CO2e', 'NOx', 'SO2', 'PM10']
    scope1_regex = re.compile('(' + ')|('.join(scope1_gases) + ')', flags=re.I)
    
    scope3_dict = { 'Purchased Goods and Services':1,
                    'Capital Goods':2,
                    'Fuel and Energy Related Activities':3,
                    'Fuel and Energy Related Activities (Market-Based)':3,
                    'Fuel and Energy Related Activities (Location-Based)':3,
                    'Upstream Transportation and Distribution':4,
                    'Transportation services':4,                # DPDHL
                    'Fuel- and energy-related activities':4,    # DPDHL
                    'Waste Generated in Operations (Large office campuses)':5,
                    'Business Travel':6,
                    'Employee Commuting':7,
                    'Upstream Leased Assets':8,
                    'Downstream Transportation and Distribution':9,
                    'Processing of Sold Products':10,
                    'Use of Sold Products':11,
                    'End of Life Treatment of Sold Products':12,
                    'Downstream Leads Assets':13,
                    'Franchises':14,
                    'Investments':15 }

    # Convert reported units to things standard in `pint`
    unit_dict = { 'trillion (10^12) MJ':'PJ', 'million MWh':'TWh', 'MW':'MW', 
                  'million tonnes CO2e':'Mt CO2e', 'tonnes CO2e':'t CO2e', 'm t CO2e':'Mt CO2e', 'MT CO2e':'Mt CO2e',
                  'million tonnes':'Mt', 'thousand tonnes':'kilot', 'tonnes':'t', 'kg':'kg', 'MT':'Mt', 'Lbs':'lbs', 'Metric Tons':'t', 
                  'tBtu':'TBtu',
                  'm liter':'M liter', 'Grams per € revenue':'Grams / EUR',
                  'Millions of m3':'1000 dam', 'm3':'m3', 'Gallons':'gal',
                  'Million Gallons':'M gallons', 'Billions of Liters':'10^9 l', 'billions of Liters':'10^9 l',}
    u2u_dict = { '%':'pct', 'Grams per € revenue':'Grams / EUR', 'revenue':'EUR', 'MM$ revenue':'1000000 revenue',
                 'short ton':'short_ton', 'No.':'[]', 'Nb':'[]', }

### Time for a Pint!

See https://github.com/IAMconsortium/units/issues/9
and https://github.com/openscm/openscm-units/issues/31

In [ ]:
import pandas as pd
import pint_pandas
from openscm_units import unit_registry

pint_pandas.PintType.ureg = u = unit_registry

one_co2 = unit_registry("CO2")
print(one_co2)

x = pd.DataFrame([[2.0,'Mt CO2']], columns=['Value', 'Unit'])
print(x)
x = x.astype({'Value': 'pint[Mt CO2]'})
print(x.Value.pint.to('t CO2'))

In [ ]:
u('Mt/1000000').to_compact()

In [ ]:
PA_ = pint_pandas.PintArray

ureg = unit_registry
Q_ = ureg.Quantity

Note that pint[unit] must be used for the Series constuctor, whereas the PintArray constructor allows the unit string or object.

```
    df = pd.DataFrame({
        "length" : pd.Series([1.,2.], dtype="pint[m]"),
        "width" : PA_([2.,3.], dtype="pint[m]"),
        "distance" : PA_([2.,3.], dtype="m"),
        "height" : PA_([2.,3.], dtype=ureg.m),
        "depth" : PA_.from_1darray_quantity(Q_([2,3],ureg.m)),
    })
```

See https://pint.readthedocs.io/en/0.18/pint-pandas.html

In [ ]:
wb = load_workbook(long_fmt_filename, data_only=True)

from itertools import islice

def long_ws_to_df(ws):
    data = ws.values
    cols = next(data)
    data = list(data)
    # idx = [r[0] for r in data]
    # data = (islice(r, 1, None) for r in data)
    
    df = pd.DataFrame(data, columns=cols)

    # The original data has topic we construct.  It is removed when writing LONG data but can be restored from SHEET_NAME
    if 'Topic' not in df.columns:
        print('Restoring Topic ' + ws.title)
        df.insert(crm.topic_col-1, 'Topic', ws.title)
    
    return df

trino_df = pd.concat([long_ws_to_df(ws) for ws in wb.worksheets])
    
len(trino_df)

In [ ]:
print(trino_df['Unit'].value_counts())
trino_df.Unit.unique()

Now create data in Trino

In [ ]:
import boto3

# Create an S3 client.  We will user later when we write out data and metadata
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [ ]:
import trino

conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
import datetime
# datetime.datetime.now()
# For now we used a fixed date so we don't fill things up needlessly
timestamp = "2008-09-03T20:56:35.450686Z"

In [ ]:
ingest_uuid = str(uuid.uuid4())

custom_meta_key_fields = 'metafields'
custom_meta_key = 'metaset'

schemaname = 'osc_corp_data'
cur.execute('create schema if not exists osc_datacommons_dev.' + schemaname)
cur.fetchall()

For osc_datacommons_dev, a trino pipeline is a parquet data stored in the S3_DEV_BUCKET
It is a 5-step process to get there from a pandas dataframe

In [ ]:
def create_trino_pipeline (s3, schemaname, tablename, timestamp, df, meta_fields, meta_content):
    global ingest_uuid
    global custom_meta_key_fields, custom_meta_key
    
    # First convert dataframe to pyarrow for type conversion and basic metadata
    table = pa.Table.from_pandas(enforce_sql_column_names(df))
    # Second, since pyarrow tables are immutable, create a new table with additional combined metadata
    if meta_fields or meta_content:
        meta_json_fields = json.dumps(meta_fields)
        meta_json = json.dumps(meta_content)
        existing_meta = table.schema.metadata
        combined_meta = {
            custom_meta_key_fields.encode(): meta_json_fields.encode(),
            custom_meta_key.encode(): meta_json.encode(),
            **existing_meta
        }
        table = table.replace_schema_metadata(combined_meta)
    # Third, convert table to parquet format (which cannot be written directly to s3)
    pq.write_table(table, '/tmp/{sname}.{tname}.{uuid}.{timestamp}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp))
    # df.to_parquet('/tmp/{sname}.{tname}.{uuid}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, index=False))
    # Fourth, put the parquet-ified data into our S3 bucket for trino.  We cannot compute parquet format directly to S3 but we can copy it once computed
    s3.upload_file(
        Bucket=os.environ['S3_DEV_BUCKET'],
        Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp),
        Filename='/tmp/{sname}.{tname}.{uuid}.{timestamp}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
    )
    # Finally, create the trino table backed by our parquet files enhanced by our metadata
    cur.execute('.'.join(['drop table if exists osc_datacommons_dev', schemaname, tablename]))
    print('dropping table: ' + tablename)
    cur.fetchall()
    
    schema = create_table_schema_pairs(df)

    tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/{uuid}/{timestamp}'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],sname=schemaname,tname=tablename,uuid=ingest_uuid,timestamp=timestamp)
    print(tabledef)

    # tables created externally may not show up immediately in cloud-beaver
    cur.execute(tabledef)
    cur.fetchall()

### Write out Report with metadata

Create the actual metadata for the source.  In this case, it is osc_corp_data.

In [ ]:
custom_meta_content = {}
metadata_text = """Title: AEP GHG and Energy Report, 2020
Description: 
Version: 2020
Release Date: 
URI: https://reports.shell.com/sustainability-report/2020/our-performance-data/greenhouse-gas-and-energy-data.html
Copyright: 
License: 
Contact: 
Citation: """

for line in metadata_text.split('\n'):
    k, v = line.split(':', 1)
    k = sql_compliant_name(k)
    custom_meta_content[k] = v

custom_meta_content['abstract'] = """Abstract text"""
custom_meta_content['name'] = 'osc_corp_data'

Create the metadata for all the fields in all the tables

Create custom meta data and key

In [ ]:
shell_df

In [ ]:
tablename = 'aep_2020'
custom_meta_fields = {}
create_trino_pipeline (s3, schemaname, tablename, timestamp, shell_df, custom_meta_fields, custom_meta_content)

Restore data and metadata

In [ ]:
# Everything below here is speculative / in process of design

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*

In [ ]:
# Create metastore structure
metastore = {'catalog':'osc_datacommons_dev',
             'schema':'aep_2020',
             'table':tablename,
             'metadata':custom_meta_content,
             'uuid':ingest_uuid}
# Create DataFrame
df_meta = pd.DataFrame(metastore)
# Print the output
df_meta

In [ ]:
help(iam_units)

In [ ]:
help(registry)